# Trusted zone

In [1]:
# ! pip install duckdb --user --upgrade --quiet
# ! pip install pyarrow --user --upgrade --quiet

In [29]:
import os
import duckdb
import numpy as np
import pandas as pd

In [28]:
def clear_datebase():
    dbFileName = os.getcwd() + '/household.duckdb'
    if(os.path.exists(dbFileName)):
        os.remove(dbFileName)
    dbFileName = os.getcwd() + '/nationalities.duckdb'
    if(os.path.exists(dbFileName)):
        os.remove(dbFileName)

clear_datebase()

## Joining tables per data source

In [35]:
def df_to_DBtable(DB,df,table):
    """
    Creates a persistent table in DuckDB from the DataFrame content.
    """
    con = duckdb.connect(DB)
    con.register(table, df)
    con.execute(f'CREATE TABLE {table} AS SELECT * FROM {table}')
    con.close()

def DBtable_to_df(table):
    """
    Converts the DB `table` in a data frame format 
    """
    repo = table[:-4]
    print(repo)
    con = duckdb.connect(f'{repo}.duckdb')
    df = con.execute(f'SELECT * FROM {table}').df()
    con.close()
    return df

def get_tables(DB):
    """
    Gets all the tables from the `DB`
    """
    con = duckdb.connect(DB)
    tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()['table_name']
    con.close()
    return tables

def pre_join(DB):
    """
    Adds a version (year) column to all `DB` tables in order to do the joining.
    Returns all the `DB` tables in data frame format.
    """
    tablesDB = get_tables(DB)
    dfs = []
    for table in tablesDB:
        year = table[-4:]
        df = DBtable_to_df(table)
        # display(df)
        df['Year'] = year
        dfs.append(df)
    return dfs

def joining(DB,table):
    """
    Joins all the `DB` tables into one and stores it in the `DB`.
    """
    dfs = pre_join(DB)
    df = pd.concat(dfs, axis=0, ignore_index=True)
    # display(df)
    df_to_DBtable(DB,df,table)

In [37]:
joining('nationalities.duckdb', 'nationalities')
joining('household.duckdb', 'household')

nationalities


,Madrid_section,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,Croacia,...,Vietnam,Yemen,Otros_Países_de_Asia,Australia,Fiji,Nueva_Zelanda,Papúa_Nueva_Guinea,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía
0,0014,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,001401,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001401001,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0029,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,002901,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4834,182101,258.0,229.0,29.0,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4835,182101001,258.0,229.0,29.0,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4836,1837,1620.0,1390.0,230.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4837,183701,1620.0,1390.0,230.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


nationalities


,Madrid_section,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,Croacia,...,Uzbekistán,Vietnam,Yemen,Otros_Países_de_Asia,Australia,Fiji,Nueva_Zelanda,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía
0,0014,68.0,67.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,001401,68.0,67.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001401001,68.0,67.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0029,4721.0,4073.0,648.0,12.0,0.0,1.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,002901,4721.0,4073.0,648.0,12.0,0.0,1.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4837,182101,262.0,235.0,27.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4838,182101001,262.0,235.0,27.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4839,1837,1734.0,1475.0,259.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4840,183701,1734.0,1475.0,259.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


nationalities


...

,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors
0,Ciudad de Madrid,1307682.0,117998.0,116863.0,126695.0,37538.0,26232.0,5328.0,163183.0,175581.0,...,25255.0,9283.0,106779.0,51993.0,8131.0,3537.0,48740.0,59406.0,622.0,422.0
1,01. Centro,69187.0,11695.0,13596.0,5057.0,2269.0,815.0,172.0,13563.0,5267.0,...,476.0,213.0,4719.0,1123.0,181.0,67.0,1898.0,2153.0,48.0,25.0
2,1001.0,568.0,75.0,93.0,52.0,17.0,6.0,5.0,97.0,74.0,...,3.0,2.0,36.0,11.0,3.0,0.0,20.0,20.0,0.0,0.0
3,1002.0,492.0,111.0,99.0,32.0,17.0,4.0,1.0,96.0,30.0,...,1.0,0.0,33.0,6.0,1.0,0.0,15.0,14.0,0.0,0.0
4,1003.0,862.0,137.0,164.0,68.0,43.0,7.0,2.0,148.0,80.0,...,10.0,3.0,50.0,16.0,7.0,0.0,28.0,37.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,21029.0,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,30.0,10.0,25.0,49.0,10.0,0.0,14.0,18.0,0.0,0.0
2461,21030.0,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,19.0,4.0,44.0,43.0,5.0,0.0,13.0,17.0,0.0,0.0
2462,21031.0,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,24.0,7.0,33.0,32.0,7.0,1.0,15.0,28.0,1.0,0.0
2463,21032.0,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,24.0,7.0,23.0,30.0,6.0,0.0,8.0,7.0,1.0,0.0


household


,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors
0,Ciudad de Madrid,1278258.0,116921.0,114806.0,124815.0,35963.0,27460.0,5488.0,156118.0,173809.0,...,25188.0,8792.0,102020.0,50724.0,7803.0,3361.0,45696.0,50463.0,513.0,342.0
1,01. Centro,66411.0,11695.0,13210.0,5173.0,2142.0,970.0,191.0,12277.0,5164.0,...,453.0,191.0,4013.0,1060.0,183.0,79.0,1718.0,1854.0,43.0,18.0
2,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,4.0,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0
3,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,2.0,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0
4,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,10.0,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,21029.0,519.0,32.0,29.0,6.0,5.0,20.0,12.0,45.0,19.0,...,27.0,10.0,29.0,49.0,7.0,2.0,8.0,12.0,0.0,0.0
2461,21030.0,566.0,37.0,46.0,29.0,11.0,19.0,6.0,68.0,52.0,...,26.0,3.0,39.0,41.0,3.0,0.0,14.0,16.0,0.0,0.0
2462,21031.0,887.0,88.0,91.0,15.0,6.0,39.0,9.0,156.0,31.0,...,24.0,6.0,28.0,21.0,6.0,2.0,10.0,16.0,0.0,0.0
2463,21032.0,627.0,65.0,71.0,17.0,4.0,27.0,13.0,82.0,23.0,...,18.0,2.0,21.0,30.0,5.0,0.0,7.0,6.0,1.0,1.0


household


,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_one_minor,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors
0,Ciudad de Madrid,1290164.0,117454.0,115691.0,126044.0,36720.0,27171.0,5491.0,158487.0,174284.0,...,25045.0,9057.0,104079.0,51419.0,8009.0,3438.0,46977.0,53887.0,552.0,405.0
1,01. Centro,67296.0,11627.0,13352.0,5117.0,2171.0,922.0,194.0,12698.0,5244.0,...,484.0,214.0,4341.0,1062.0,184.0,84.0,1711.0,1934.0,44.0,33.0
2,1001.0,559.0,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,...,3.0,1.0,35.0,11.0,4.0,2.0,13.0,17.0,0.0,1.0
3,1002.0,471.0,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,...,4.0,0.0,41.0,11.0,1.0,0.0,13.0,12.0,0.0,0.0
4,1003.0,855.0,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,...,16.0,2.0,54.0,16.0,5.0,1.0,20.0,35.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,21029.0,526.0,35.0,33.0,7.0,5.0,21.0,9.0,36.0,24.0,...,33.0,11.0,25.0,51.0,8.0,2.0,9.0,16.0,0.0,0.0
2461,21030.0,563.0,33.0,59.0,26.0,8.0,16.0,4.0,70.0,49.0,...,19.0,3.0,36.0,43.0,5.0,0.0,12.0,21.0,0.0,0.0
2462,21031.0,948.0,99.0,94.0,14.0,7.0,41.0,8.0,155.0,35.0,...,23.0,6.0,29.0,25.0,6.0,0.0,17.0,23.0,0.0,0.0
2463,21032.0,658.0,72.0,67.0,18.0,6.0,33.0,13.0,94.0,25.0,...,18.0,3.0,22.0,27.0,7.0,0.0,6.0,9.0,1.0,2.0


### Verify joined tables per data source

#### nationalities 

In [40]:
# nationalities joined
con = duckdb.connect('nationalities.duckdb')
view_nationalities = con.execute(f'SELECT * FROM nationalities').df()
display(view_nationalities)
con.close()

,Madrid_section,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,Croacia,...,Papúa_Nueva_Guinea,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía,Year,Belarús,Mónaco,Santa_Sede,República_Democrática_del_Congo,Samoa
0,0014,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
1,001401,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
2,001401001,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
3,0029,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
4,002901,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14518,182101,253.0,226.0,27.0,0.0,0.0,1.0,5.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14519,182101001,253.0,226.0,27.0,0.0,0.0,1.0,5.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14520,1837,1658.0,1413.0,245.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14521,183701,1658.0,1413.0,245.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0


#### household

In [39]:
# households joined
con = duckdb.connect('household.duckdb')
view_household = con.execute(f'SELECT * FROM household').df()
display(view_household)
con.close()

,section,total,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,Ciudad de Madrid,1307682.0,117998.0,116863.0,126695.0,37538.0,26232.0,5328.0,163183.0,175581.0,...,9283.0,106779.0,51993.0,8131.0,3537.0,48740.0,59406.0,622.0,422.0,2020
1,01. Centro,69187.0,11695.0,13596.0,5057.0,2269.0,815.0,172.0,13563.0,5267.0,...,213.0,4719.0,1123.0,181.0,67.0,1898.0,2153.0,48.0,25.0,2020
2,1001.0,568.0,75.0,93.0,52.0,17.0,6.0,5.0,97.0,74.0,...,2.0,36.0,11.0,3.0,0.0,20.0,20.0,0.0,0.0,2020
3,1002.0,492.0,111.0,99.0,32.0,17.0,4.0,1.0,96.0,30.0,...,0.0,33.0,6.0,1.0,0.0,15.0,14.0,0.0,0.0,2020
4,1003.0,862.0,137.0,164.0,68.0,43.0,7.0,2.0,148.0,80.0,...,3.0,50.0,16.0,7.0,0.0,28.0,37.0,0.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,21029.0,526.0,35.0,33.0,7.0,5.0,21.0,9.0,36.0,24.0,...,11.0,25.0,51.0,8.0,2.0,9.0,16.0,0.0,0.0,2019
7391,21030.0,563.0,33.0,59.0,26.0,8.0,16.0,4.0,70.0,49.0,...,3.0,36.0,43.0,5.0,0.0,12.0,21.0,0.0,0.0,2019
7392,21031.0,948.0,99.0,94.0,14.0,7.0,41.0,8.0,155.0,35.0,...,6.0,29.0,25.0,6.0,0.0,17.0,23.0,0.0,0.0,2019
7393,21032.0,658.0,72.0,67.0,18.0,6.0,33.0,13.0,94.0,25.0,...,3.0,22.0,27.0,7.0,0.0,6.0,9.0,1.0,2.0,2019
